In [1]:
import random
import shapely
import pointpats
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import numpy as np
import time
from scipy.optimize import differential_evolution
import random
from scipy.optimize import dual_annealing
from scipy.stats import rice
import csv


In [ ]:
#Differential Annealing

def obj_f(x):
    y = np.sort([x[10], x[11], x[12]])
    b_ratio = np.array([y[0], y[1] - y[0], y[2] - y[1], 1 - y[2]])
    # print(f"==>> b_ratio: {b_ratio}")

    link = x[13:23]
    # print(f"==>> link: {link}")

    def convert_X(x):
        disc_x = np.floor(link).astype(int)
        valid_values = {0, 1, 2, 3}
        # print(f"==>> disc_x: {disc_x}")
        # Check for valid values and handle invalid cases
        if set(disc_x) == valid_values:
            list_oneD = disc_x.tolist()
            # print(f"==>> list_oneD: {list_oneD}")
            return list_oneD
        else:
            # Handle invalid cases, for example, return a valid default value
            return [0, 1, 2, 3]
    def X_to_Matrix(y):
        if y is None:
            return None

        num_rows = max(y) + 1
        num_cols = len(y)

        # Initialize matrix with zeros
        matrix = np.zeros((num_rows, num_cols), dtype=int)

        for col_index, row_index in enumerate(y):
            # Check for out-of-bounds indices and handle appropriately
            if 0 <= row_index < num_rows:
                matrix[row_index, col_index] = 1
            else:
                # Handle out-of-bounds indices, for example, skip or set to a default value
                pass

        return matrix

    def check(matrix):
        if matrix is None:
            return False

        # Check for all() and any() conditions separately
        col_conditions = [np.sum(matrix[:, col] == 1) for col in range(matrix.shape[1])]
        row_conditions = np.sum(matrix, axis=1)

        return all(condition for condition in col_conditions) and all(row > 0 for row in row_conditions)

    def matrix_to_set(matrix):
        if check(matrix):
            set_result = [list(np.where(row == 1)[0]) for row in matrix]
            # print(f"==>> set_result: {set_result}")
            return set_result
        else:
            # Handle invalid matrix, for example, return an empty list or other default value
            return []

    # Usage in your main function
    x_converted = convert_X(x[13:23])
    # print(f"==>> x_converted: {convert_X(x[13:23])}")
    if not x_converted:
        return np.inf  # Return a large value to penalize invalid solutions

    matrix_x = X_to_Matrix(x_converted)
    # print(f"==>> matrix_x: {matrix_x}")
    if matrix_x is None:
        return np.inf  # Return a large value to penalize invalid solutions

    set_x = matrix_to_set(matrix_x)
    if not set_x:
        return np.inf  # Return a large value to penalize invalid solutions


    return -obj_function(hap_xyz=np.array([x[0], x[1], HAP_Z_MAX]), uav_xyz=np.array([[x[2], x[3], UAV_Z_MAX], [x[4], x[5], UAV_Z_MAX]]),
                            p_leo=x[6], p_hap=x[7], p_uav1=x[8], p_uav2=x[9],
                            b_s=b_ratio[0],
                            b_h=b_ratio[1],
                            b_u1=b_ratio[2],
                            b_u2=b_ratio[3],
                            iot_select_leo=set_x[0],
                            iot_select_hap=set_x[1],
                            iot_select_uav1=set_x[2],
                            iot_select_uav2=set_x[3]
    )
bounds = [(-2000, 2000), (-2000, 2000), (-2000, 2000), (-2000, 2000), (-2000, 2000), (-2000, 2000),
        (0, 1), (0, 1), (0, 1), (0, 1),
        (0, 1), (0, 1), (0, 1),
        (0,4),(0,4),(0,4),(0,4),(0,4),(0,4),(0,4),(0,4),(0,4),(0,4)]
results = {'fun': [], 'x': []}

# Define the callback function to print results at each iteration
def callback(xk, f, context):
    results['fun'].append(f)
    results['x'].append(xk)
    print(f"Iteration {len(results['x']):<3}: f(x) = {f:.4f}, x = {xk}")
    # print(f"Iteration {len(results['x']):<3}: f(x) = {f:.4f}")
# result = dual_annealing(obj_f, bounds, callback=callback, maxiter=100)
try:
    result = dual_annealing(obj_f, bounds, callback=callback, maxiter=2000 )
except Exception as e:
    print(f"An error occurred: {e}")

# Initialize an empty dictionary to store results
print("\nOptimization Result:")
print(f"Final f(x) = {result['fun']:.4f}")
print(f"Final x = {result['x']}")
